# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Procesamiento del Corpus:
    * Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).

2. Procesamiento de las Consultas:
    * Definir las consultas proporcionadas.
    * Extraer las palabras clave de cada consulta.

3. Cálculo de Relevancia:
    * Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
    * Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

4. Ranking de Documentos:
    * Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
    * Mostrar el ranking de documentos para cada consulta.

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import re
from collections import Counter

In [ ]:
# Cargar y parsear el archivo XML
tree = ET.parse('/kaggle/input/03ranking-corpus/03ranking_corpus.xml')
root = tree.getroot()

# Crear una lista para almacenar los documentos con sus metadatos
documents = []

# Iterar a través de cada elemento 'document' en el archivo XML
for doc in root.findall('document'):
    doc_id = doc.get('id')
    title = doc.find('title').text
    keywords = doc.find('keywords').text
    author = doc.find('author').text
    date = doc.find('date').text
    
    documents.append({
        'id': doc_id,
        'title': title,
        'keywords': keywords,
        'author': author,
        'date': date
    })

In [ ]:
# Mostrar el resultado
for document in documents:
    print(f"ID: {document['id']}")
    print(f"Title: {document['title']}")
    print(f"Keywords: {document['keywords']}")
    print(f"Author: {document['author']}")
    print(f"Date: {document['date']}")
    print("-" * 40)

### Procesamiento de las Consultas:

- Definir las consultas proporcionadas.
- Extraer las palabras clave de cada consulta.

In [ ]:
# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

In [ ]:
# Mostrar las palabras clave extraídas
for query_id, query_text in queries.items():
    # Procesar el texto de la consulta
    keywords = process_text(query_text)
    print(f"Consulta {query_id}: {keywords}")

### Cálculo de Relevancia:
- Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.
- Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.

In [ ]:
# Función de similitud coseno
def cosine_similarity(query, doc):
    query_tokens = process_text(query)
    doc_tokens = process_text(doc)

    # Crear el vocabulario (todas las palabras únicas en la consulta y el documento)
    vocabulary = list(query_tokens | doc_tokens)  # Unión de los conjuntos

    # Crear los vectores de frecuencia (binarios)
    query_vector = [1 if word in query_tokens else 0 for word in vocabulary]
    doc_vector = [1 if word in doc_tokens else 0 for word in vocabulary]

    # Calcular el producto punto y las normas
    dot_product = np.dot(query_vector, doc_vector)
    norm_query = np.linalg.norm(query_vector)
    norm_doc = np.linalg.norm(doc_vector)

    # Similitud coseno
    if norm_query == 0 or norm_doc == 0:
        return 0  # No hay similitud si uno de los vectores es cero
    return dot_product / (norm_query * norm_doc)

In [ ]:
# Función de similitud de Jaccard
def jaccard_similarity(query, doc):
    query_tokens = process_text(query)
    doc_tokens = process_text(doc)

    # Intersección y unión de los conjuntos
    intersection = query_tokens & doc_tokens
    union = query_tokens | doc_tokens
    return len(intersection) / len(union)

In [ ]:
# Calcular las puntuaciones de relevancia para cada documento respecto a cada consulta
for query_id, query_text in queries.items():
    print(f"Consulta {query_id}: {query_text}")
    
    for doc in documents:
        doc_text = doc['title'] + ' ' + doc['keywords'] 
        cosine_score = cosine_similarity(query_text, doc_text) # Aplicamos el método para calcular la similaridad del coseno
        jaccard_score = jaccard_similarity(query_text, doc_text) # Aplicamos el método para calcular la similaridad de Jaccard
        
        print(f"  Documento {doc['id']}: {doc['title']}")
        print(f"    Similitud Coseno: {cosine_score:.4f}")
        print(f"    Similitud de Jaccard: {jaccard_score:.4f}")
    print("\n")

### Ranking de Documentos:

- Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
- Mostrar el ranking de documentos para cada consulta.

In [ ]:
# Se define k para limitar el número de documentos en el ranking
k = 7

# Diccionario para almacenar los rankings de relevancia de cada consulta
rankings = {}

# Iterar sobre cada consulta para calcular las similitudes y generar el ranking
for query_id, query_text in queries.items():
    # Lista para almacenar los documentos con su puntuación de relevancia
    relevance_scores = []

    # Iterar sobre cada documento en el corpus
    for doc in documents:
        # Concatenar título y palabras clave para formar el contenido del documento
        doc_text = doc['title'] + ' ' + doc['keywords']
        
        # Calcular la similitud coseno y jaccard utilizando los métodos ya definidos
        cosine_score = cosine_similarity(query_text, doc_text)
        jaccard_score = jaccard_similarity(query_text, doc_text)
        
        # Calculo de relevancia
        relevance_score = (cosine_score + jaccard_score) / 2
        
        # Agregar el documento con su relevancia a la lista
        relevance_scores.append({
            'doc_id': doc['id'],
            'title': doc['title'],
            'cosine_score': cosine_score,
            'jaccard_score': jaccard_score,
            'relevance_score': relevance_score
        })

    # Ordenar los documentos por su puntuación de relevancia (de mayor a menor)
    relevance_scores = sorted(relevance_scores, key=lambda x: x['relevance_score'], reverse=True)

    # Almacenar solo los k documentos más relevantes para esta consulta
    rankings[query_id] = relevance_scores[:k]

In [ ]:
# Mostrar el ranking de documentos para cada consulta
for query_id, ranked_docs in rankings.items():
    print(f"\nRanking para la Consulta {query_id}: {queries[query_id]}")
    for rank, doc in enumerate(ranked_docs, start=1):
        print(f"  {rank}. Documento {doc['doc_id']}: {doc['title']}")
        print(f"     Similitud Coseno: {doc['cosine_score']:.4f}")
        print(f"     Similitud de Jaccard: {doc['jaccard_score']:.4f}")
        print(f"     Puntuación de Relevancia Total: {doc['relevance_score']:.4f}")